In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import collections
from collections import OrderedDict
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

####### Macro Cases ######
dfn1_macro_fused_probempi = pd.read_csv("n1_fused_macro_probmpi.csv")
dfn1_macro_fused_probempi=dfn1_macro_fused_probempi.sort_values(by=['Dirc_Data_Mov'])
dfn1_macro_fused_nonprobempi = pd.read_csv("n1_fused_macro_nonprobmpi.csv")
dfn1_macro_fused_nonprobempi=dfn1_macro_fused_nonprobempi.sort_values(by=['Dirc_Data_Mov'])
dfn1_macro_newgeo_probempi = pd.read_csv("n1_newgeo_macro_probmpi.csv")
dfn1_macro_newgeo_nonprobempi = pd.read_csv("n1_newgeo_macro_nonprobmpi.csv")
dfn1_macro_nonfused_probempi = pd.read_csv("n1_nonfused_macro_probmpi.csv")
dfn1_macro_nonfused_nonprobempi = pd.read_csv("n1_nonfused_macro_nonprobmpi.csv")

####### AVX Cases ######
dfn1_avx_fused_probmpi = pd.read_csv("n1_fused_avx_probmpi.csv")
dfn1_avx_fused_nonprobmpi = pd.read_csv("n1_fused_avx_nonprobmpi.csv")
dfn1_avx_newgeo_probempi = pd.read_csv("n1_newgeo_avx_probmpi.csv")
dfn1_avx_newgeo_nonprobempi = pd.read_csv("n1_newgeo_avx_nonprobmpi.csv")
dfn1_avx_nonfused_probempi = pd.read_csv("n1_nonfused_avx_probmpi.csv")
dfn1_avx_nonfused_nonprobempi = pd.read_csv("n1_nonfused_avx_nonprobmpi.csv")

####### VECT Cases ######
dfn1_vect_fused_probmpi = pd.read_csv("n1_fused_vect_probmpi.csv")
dfn1_vect_fused_nonprobmpi = pd.read_csv("n1_fused_vect_nonprobmpi.csv")
dfn1_vect_newgeo_probempi = pd.read_csv("n1_newgeo_vect_probmpi.csv")
dfn1_vect_newgeo_nonprobempi = pd.read_csv("n1_newgeo_vect_nonprobmpi.csv")
dfn1_vect_nonfused_probempi = pd.read_csv("n1_nonfused_vect_probmpi.csv")
dfn1_vect_nonfused_nonprobempi = pd.read_csv("n1_nonfused_vect_nonprobmpi.csv")



# HIPPO MACRO CASES

## 1) Fused_Macro_Probempi

### Adding Columns

In [3]:
dfn1_macro_fused_probempi["N_Nodes"] = 1
dfn1_macro_fused_probempi['NPX'] = (dfn1_macro_fused_probempi.GLB_X//dfn1_macro_fused_probempi.LOC_X)
dfn1_macro_fused_probempi['NPY'] = (dfn1_macro_fused_probempi.GLB_Y//dfn1_macro_fused_probempi.LOC_Y)
dfn1_macro_fused_probempi['NPZ'] = (dfn1_macro_fused_probempi.GLB_Z//dfn1_macro_fused_probempi.LOC_Z)
dfn1_macro_fused_probempi['NPT'] = (dfn1_macro_fused_probempi.GLB_T//dfn1_macro_fused_probempi.LOC_T)
dfn1_macro_fused_probempi["Total_proc"] = dfn1_macro_fused_probempi["omp_thrd"]* dfn1_macro_fused_probempi.MPI_size
# dfn1_macro_fused_probempi.columns

### Data Summary

In [4]:
dfn1_macro_fused_probempi_summary = dfn1_macro_fused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_macro_fused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,34
3,64,2,8,2,2,2,128,16
4,128,1,16,2,2,2,128,6


### Case Setup

In [5]:
dfn1MacFusProb_8mpi_16omp = dfn1_macro_fused_probempi[(dfn1_macro_fused_probempi.omp_thrd == 16) & (dfn1_macro_fused_probempi.GLB_T/dfn1_macro_fused_probempi.LOC_T == 1) & (dfn1_macro_fused_probempi.GLB_X/dfn1_macro_fused_probempi.LOC_X == 2) & (dfn1_macro_fused_probempi.GLB_Y/dfn1_macro_fused_probempi.LOC_Y == 2)& (dfn1_macro_fused_probempi.GLB_Z/dfn1_macro_fused_probempi.LOC_Z == 2)]
dfn1MacFusProb_16mpi_8omp = dfn1_macro_fused_probempi[(dfn1_macro_fused_probempi.omp_thrd == 8) & (dfn1_macro_fused_probempi.GLB_T/dfn1_macro_fused_probempi.LOC_T == 2) & (dfn1_macro_fused_probempi.GLB_X/dfn1_macro_fused_probempi.LOC_X == 2) & (dfn1_macro_fused_probempi.GLB_Y/dfn1_macro_fused_probempi.LOC_Y == 2)& (dfn1_macro_fused_probempi.GLB_Z/dfn1_macro_fused_probempi.LOC_Z == 2)]
dfn1MacFusProb_32mpi_4omp = dfn1_macro_fused_probempi[(dfn1_macro_fused_probempi.omp_thrd == 4) & (dfn1_macro_fused_probempi.GLB_T/dfn1_macro_fused_probempi.LOC_T == 4) & (dfn1_macro_fused_probempi.GLB_X/dfn1_macro_fused_probempi.LOC_X == 2) & (dfn1_macro_fused_probempi.GLB_Y/dfn1_macro_fused_probempi.LOC_Y == 2)& (dfn1_macro_fused_probempi.GLB_Z/dfn1_macro_fused_probempi.LOC_Z == 2)]
dfn1MacFusProb_64mpi_2omp = dfn1_macro_fused_probempi[(dfn1_macro_fused_probempi.omp_thrd == 2) & (dfn1_macro_fused_probempi.GLB_T/dfn1_macro_fused_probempi.LOC_T == 8) & (dfn1_macro_fused_probempi.GLB_X/dfn1_macro_fused_probempi.LOC_X == 2) & (dfn1_macro_fused_probempi.GLB_Y/dfn1_macro_fused_probempi.LOC_Y == 2)& (dfn1_macro_fused_probempi.GLB_Z/dfn1_macro_fused_probempi.LOC_Z == 2)]
dfn1MacFusProb_128mpi_1omp = dfn1_macro_fused_probempi[(dfn1_macro_fused_probempi.omp_thrd == 1) & (dfn1_macro_fused_probempi.GLB_T/dfn1_macro_fused_probempi.LOC_T == 16) & (dfn1_macro_fused_probempi.GLB_X/dfn1_macro_fused_probempi.LOC_X == 2) & (dfn1_macro_fused_probempi.GLB_Y/dfn1_macro_fused_probempi.LOC_Y == 2)& (dfn1_macro_fused_probempi.GLB_Z/dfn1_macro_fused_probempi.LOC_Z == 2)]


## 2) Fused_Macro_nonprobempi

### Adding Columns

In [6]:
dfn1_macro_fused_nonprobempi["N_Nodes"] = 1
dfn1_macro_fused_nonprobempi['NPX'] = (dfn1_macro_fused_nonprobempi.GLB_X//dfn1_macro_fused_nonprobempi.LOC_X)
dfn1_macro_fused_nonprobempi['NPY'] = (dfn1_macro_fused_nonprobempi.GLB_Y//dfn1_macro_fused_nonprobempi.LOC_Y)
dfn1_macro_fused_nonprobempi['NPZ'] = (dfn1_macro_fused_nonprobempi.GLB_Z//dfn1_macro_fused_nonprobempi.LOC_Z)
dfn1_macro_fused_nonprobempi['NPT'] = (dfn1_macro_fused_nonprobempi.GLB_T//dfn1_macro_fused_nonprobempi.LOC_T)
dfn1_macro_fused_nonprobempi["Total_proc"] = dfn1_macro_fused_nonprobempi["omp_thrd"]* dfn1_macro_fused_nonprobempi.MPI_size
# dfn1_macro_fused_nonprobempi.columns

### Data Summary

In [7]:
dfn1_macro_fused_nonprobempi_summary =dfn1_macro_fused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_macro_fused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,70
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


## Cases Setup

In [8]:
dfn1MacFusNonProb_8mpi_16omp = dfn1_macro_fused_nonprobempi[(dfn1_macro_fused_nonprobempi.omp_thrd == 16) & (dfn1_macro_fused_nonprobempi.GLB_T/dfn1_macro_fused_nonprobempi.LOC_T == 1) & (dfn1_macro_fused_nonprobempi.GLB_X/dfn1_macro_fused_nonprobempi.LOC_X == 2) & (dfn1_macro_fused_nonprobempi.GLB_Y/dfn1_macro_fused_nonprobempi.LOC_Y == 2)& (dfn1_macro_fused_nonprobempi.GLB_Z/dfn1_macro_fused_nonprobempi.LOC_Z == 2)]
dfn1MacFusNonProb_16mpi_8omp = dfn1_macro_fused_nonprobempi[(dfn1_macro_fused_nonprobempi.omp_thrd == 8) & (dfn1_macro_fused_nonprobempi.GLB_T/dfn1_macro_fused_nonprobempi.LOC_T == 2) & (dfn1_macro_fused_nonprobempi.GLB_X/dfn1_macro_fused_nonprobempi.LOC_X == 2) & (dfn1_macro_fused_nonprobempi.GLB_Y/dfn1_macro_fused_nonprobempi.LOC_Y == 2)& (dfn1_macro_fused_nonprobempi.GLB_Z/dfn1_macro_fused_nonprobempi.LOC_Z == 2)]
dfn1MacFusnonProb_32mpi_4omp = dfn1_macro_fused_nonprobempi[(dfn1_macro_fused_nonprobempi.omp_thrd == 4) & (dfn1_macro_fused_nonprobempi.GLB_T/dfn1_macro_fused_nonprobempi.LOC_T == 4) & (dfn1_macro_fused_nonprobempi.GLB_X/dfn1_macro_fused_nonprobempi.LOC_X == 2) & (dfn1_macro_fused_nonprobempi.GLB_Y/dfn1_macro_fused_nonprobempi.LOC_Y == 2)& (dfn1_macro_fused_nonprobempi.GLB_Z/dfn1_macro_fused_nonprobempi.LOC_Z == 2)]
dfn1MacFusNonProb_64mpi_2omp = dfn1_macro_fused_nonprobempi[(dfn1_macro_fused_nonprobempi.omp_thrd == 2) & (dfn1_macro_fused_nonprobempi.GLB_T/dfn1_macro_fused_nonprobempi.LOC_T == 8) & (dfn1_macro_fused_nonprobempi.GLB_X/dfn1_macro_fused_nonprobempi.LOC_X == 2) & (dfn1_macro_fused_nonprobempi.GLB_Y/dfn1_macro_fused_nonprobempi.LOC_Y == 2)& (dfn1_macro_fused_nonprobempi.GLB_Z/dfn1_macro_fused_nonprobempi.LOC_Z == 2)]
dfn1MacFusnonProb_128mpi_1omp = dfn1_macro_fused_nonprobempi[(dfn1_macro_fused_nonprobempi.omp_thrd == 1) & (dfn1_macro_fused_nonprobempi.GLB_T/dfn1_macro_fused_nonprobempi.LOC_T == 16) & (dfn1_macro_fused_nonprobempi.GLB_X/dfn1_macro_fused_nonprobempi.LOC_X == 2) & (dfn1_macro_fused_nonprobempi.GLB_Y/dfn1_macro_fused_nonprobempi.LOC_Y == 2)& (dfn1_macro_fused_nonprobempi.GLB_Z/dfn1_macro_fused_nonprobempi.LOC_Z == 2)]


## GFLOPS - Fused Macro: Probempi vs NonProbempi

In [9]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn1MacFusnonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacFusnonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacFusnonProb_128mpi_1omp.GFLOPS*(dfn1MacFusnonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusnonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn1MacFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacFusProb_128mpi_1omp.GFLOPS*(dfn1MacFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn1MacFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacFusNonProb_64mpi_2omp.GFLOPS*(dfn1MacFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacFusProb_64mpi_2omp.GFLOPS*(dfn1MacFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn1MacFusnonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacFusnonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacFusnonProb_32mpi_4omp.GFLOPS*(dfn1MacFusnonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusnonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacFusProb_32mpi_4omp.GFLOPS*(dfn1MacFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn1MacFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacFusNonProb_16mpi_8omp.GFLOPS*(dfn1MacFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacFusProb_16mpi_8omp.GFLOPS*(dfn1MacFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn1MacFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacFusNonProb_8mpi_16omp.GFLOPS*(dfn1MacFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacFusProb_8mpi_16omp.GFLOPS*(dfn1MacFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))



# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

# fig.update_yaxes(color="black")
# fig.update_xaxes(color="red")
fig.update_xaxes(categoryorder="category ascending")

fig.update_layout(title='"Fused-Macro (1-Node): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

# BANDWIDTH

In [10]:
fig = go.Figure() 

fig.add_trace(go.Scatter(x=(dfn1MacFusnonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacFusnonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacFusnonProb_128mpi_1omp.BAND*(dfn1MacFusnonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusnonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


fig.add_trace(go.Scatter(x=(dfn1MacFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacFusProb_128mpi_1omp.BAND*(dfn1MacFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

# case 2
fig.add_trace(go.Scatter(x=(dfn1MacFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacFusNonProb_64mpi_2omp.BAND*(dfn1MacFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacFusProb_64mpi_2omp.BAND*(dfn1MacFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

# case 3
fig.add_trace(go.Scatter(x=(dfn1MacFusnonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacFusnonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacFusnonProb_32mpi_4omp.BAND*(dfn1MacFusnonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusnonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacFusProb_32mpi_4omp.BAND*(dfn1MacFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# case 4
fig.add_trace(go.Scatter(x=(dfn1MacFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacFusNonProb_16mpi_8omp.BAND*(dfn1MacFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacFusProb_16mpi_8omp.BAND*(dfn1MacFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))

# case 5

fig.add_trace(go.Scatter(x=(dfn1MacFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacFusNonProb_8mpi_16omp.BAND*(dfn1MacFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacFusProb_8mpi_16omp.BAND*(dfn1MacFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))


# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=300
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Fused-Macro (1-Node): probempi vs nonprobempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 


##  3) Nonfused_Macro_Probempi

### Adding Columns

In [11]:
dfn1_macro_nonfused_probempi["N_Nodes"] = 1
dfn1_macro_nonfused_probempi['NPX'] = (dfn1_macro_nonfused_probempi.GLB_X//dfn1_macro_nonfused_probempi.LOC_X)
dfn1_macro_nonfused_probempi['NPY'] = (dfn1_macro_nonfused_probempi.GLB_Y//dfn1_macro_nonfused_probempi.LOC_Y)
dfn1_macro_nonfused_probempi['NPZ'] = (dfn1_macro_nonfused_probempi.GLB_Z//dfn1_macro_nonfused_probempi.LOC_Z)
dfn1_macro_nonfused_probempi['NPT'] = (dfn1_macro_nonfused_probempi.GLB_T//dfn1_macro_nonfused_probempi.LOC_T)
dfn1_macro_nonfused_probempi["Total_proc"] = dfn1_macro_nonfused_probempi["omp_thrd"]* dfn1_macro_nonfused_probempi.MPI_size
# dfn1_macro_nonfused_probempi.columns

### Data Summary 

In [12]:
dfn1_macro_nonfused_probempi_summary = dfn1_macro_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_macro_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,70
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [13]:
dfn1MacNonFusProb_8mpi_16omp = dfn1_macro_nonfused_probempi[(dfn1_macro_nonfused_probempi.omp_thrd == 16) & (dfn1_macro_nonfused_probempi.GLB_T/dfn1_macro_nonfused_probempi.LOC_T == 1) & (dfn1_macro_nonfused_probempi.GLB_X/dfn1_macro_nonfused_probempi.LOC_X == 2) & (dfn1_macro_nonfused_probempi.GLB_Y/dfn1_macro_nonfused_probempi.LOC_Y == 2)& (dfn1_macro_nonfused_probempi.GLB_Z/dfn1_macro_nonfused_probempi.LOC_Z == 2)]
dfn1MacNonFusProb_16mpi_8omp = dfn1_macro_nonfused_probempi[(dfn1_macro_nonfused_probempi.omp_thrd == 8) & (dfn1_macro_nonfused_probempi.GLB_T/dfn1_macro_nonfused_probempi.LOC_T == 2) & (dfn1_macro_nonfused_probempi.GLB_X/dfn1_macro_nonfused_probempi.LOC_X == 2) & (dfn1_macro_nonfused_probempi.GLB_Y/dfn1_macro_nonfused_probempi.LOC_Y == 2)& (dfn1_macro_nonfused_probempi.GLB_Z/dfn1_macro_nonfused_probempi.LOC_Z == 2)]
dfn1MacNonFusProb_32mpi_4omp = dfn1_macro_nonfused_probempi[(dfn1_macro_nonfused_probempi.omp_thrd == 4) & (dfn1_macro_nonfused_probempi.GLB_T/dfn1_macro_nonfused_probempi.LOC_T == 4) & (dfn1_macro_nonfused_probempi.GLB_X/dfn1_macro_nonfused_probempi.LOC_X == 2) & (dfn1_macro_nonfused_probempi.GLB_Y/dfn1_macro_nonfused_probempi.LOC_Y == 2)& (dfn1_macro_nonfused_probempi.GLB_Z/dfn1_macro_nonfused_probempi.LOC_Z == 2)]
dfn1MacNonFusProb_64mpi_2omp = dfn1_macro_nonfused_probempi[(dfn1_macro_nonfused_probempi.omp_thrd == 2) & (dfn1_macro_nonfused_probempi.GLB_T/dfn1_macro_nonfused_probempi.LOC_T == 8) & (dfn1_macro_nonfused_probempi.GLB_X/dfn1_macro_nonfused_probempi.LOC_X == 2) & (dfn1_macro_nonfused_probempi.GLB_Y/dfn1_macro_nonfused_probempi.LOC_Y == 2)& (dfn1_macro_nonfused_probempi.GLB_Z/dfn1_macro_nonfused_probempi.LOC_Z == 2)]
dfn1MacNonFusProb_128mpi_1omp = dfn1_macro_nonfused_probempi[(dfn1_macro_nonfused_probempi.omp_thrd == 1) & (dfn1_macro_nonfused_probempi.GLB_T/dfn1_macro_nonfused_probempi.LOC_T == 16) & (dfn1_macro_nonfused_probempi.GLB_X/dfn1_macro_nonfused_probempi.LOC_X == 2) & (dfn1_macro_nonfused_probempi.GLB_Y/dfn1_macro_nonfused_probempi.LOC_Y == 2)& (dfn1_macro_nonfused_probempi.GLB_Z/dfn1_macro_nonfused_probempi.LOC_Z == 2)]


## 4) Nonfused_Macro_NonProbempi

### Adding Columns

In [14]:
dfn1_macro_nonfused_nonprobempi["N_Nodes"] = 1
dfn1_macro_nonfused_nonprobempi['NPX'] = (dfn1_macro_nonfused_nonprobempi.GLB_X//dfn1_macro_nonfused_nonprobempi.LOC_X)
dfn1_macro_nonfused_nonprobempi['NPY'] = (dfn1_macro_nonfused_nonprobempi.GLB_Y//dfn1_macro_nonfused_nonprobempi.LOC_Y)
dfn1_macro_nonfused_nonprobempi['NPZ'] = (dfn1_macro_nonfused_nonprobempi.GLB_Z//dfn1_macro_nonfused_nonprobempi.LOC_Z)
dfn1_macro_nonfused_nonprobempi['NPT'] = (dfn1_macro_nonfused_nonprobempi.GLB_T//dfn1_macro_nonfused_nonprobempi.LOC_T)
dfn1_macro_nonfused_nonprobempi["Total_proc"] = dfn1_macro_nonfused_nonprobempi["omp_thrd"]* dfn1_macro_nonfused_nonprobempi.MPI_size
# dfn1_macro_nonfused_nonprobempi.columns

### Data Summary

In [15]:
dfn1_macro_nonfused_nonprobempi_summary = dfn1_macro_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_macro_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,70
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [16]:
dfn1MacNonFusNonProb_8mpi_16omp = dfn1_macro_nonfused_nonprobempi[(dfn1_macro_nonfused_nonprobempi.omp_thrd == 16) & (dfn1_macro_nonfused_nonprobempi.GLB_T/dfn1_macro_nonfused_nonprobempi.LOC_T == 1) & (dfn1_macro_nonfused_nonprobempi.GLB_X/dfn1_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn1_macro_nonfused_nonprobempi.GLB_Y/dfn1_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_macro_nonfused_nonprobempi.GLB_Z/dfn1_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn1MacNonFusNonProb_16mpi_8omp = dfn1_macro_nonfused_nonprobempi[(dfn1_macro_nonfused_nonprobempi.omp_thrd == 8) & (dfn1_macro_nonfused_nonprobempi.GLB_T/dfn1_macro_nonfused_nonprobempi.LOC_T == 2) & (dfn1_macro_nonfused_nonprobempi.GLB_X/dfn1_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn1_macro_nonfused_nonprobempi.GLB_Y/dfn1_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_macro_nonfused_nonprobempi.GLB_Z/dfn1_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn1MacNonFusNonProb_32mpi_4omp = dfn1_macro_nonfused_nonprobempi[(dfn1_macro_nonfused_nonprobempi.omp_thrd == 4) & (dfn1_macro_nonfused_nonprobempi.GLB_T/dfn1_macro_nonfused_nonprobempi.LOC_T == 4) & (dfn1_macro_nonfused_nonprobempi.GLB_X/dfn1_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn1_macro_nonfused_nonprobempi.GLB_Y/dfn1_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_macro_nonfused_nonprobempi.GLB_Z/dfn1_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn1MacNonFusNonProb_64mpi_2omp = dfn1_macro_nonfused_nonprobempi[(dfn1_macro_nonfused_nonprobempi.omp_thrd == 2) & (dfn1_macro_nonfused_nonprobempi.GLB_T/dfn1_macro_nonfused_nonprobempi.LOC_T == 8) & (dfn1_macro_nonfused_nonprobempi.GLB_X/dfn1_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn1_macro_nonfused_nonprobempi.GLB_Y/dfn1_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_macro_nonfused_nonprobempi.GLB_Z/dfn1_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn1MacNonFusNonProb_128mpi_1omp = dfn1_macro_nonfused_nonprobempi[(dfn1_macro_nonfused_nonprobempi.omp_thrd == 1) & (dfn1_macro_nonfused_nonprobempi.GLB_T/dfn1_macro_nonfused_nonprobempi.LOC_T == 16) & (dfn1_macro_nonfused_nonprobempi.GLB_X/dfn1_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn1_macro_nonfused_nonprobempi.GLB_Y/dfn1_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_macro_nonfused_nonprobempi.GLB_Z/dfn1_macro_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - Macro nonfused: Probempi vs Nonprobempi

In [17]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_128mpi_1omp.GFLOPS*(dfn1MacNonFusNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacNonFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_128mpi_1omp.GFLOPS*(dfn1MacNonFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_64mpi_2omp.GFLOPS*(dfn1MacNonFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacNonFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_64mpi_2omp.GFLOPS*(dfn1MacNonFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_32mpi_4omp.GFLOPS*(dfn1MacNonFusNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacNonFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_32mpi_4omp.GFLOPS*(dfn1MacNonFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_16mpi_8omp.GFLOPS*(dfn1MacNonFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacNonFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_16mpi_8omp.GFLOPS*(dfn1MacNonFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))



# Case 5
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_8mpi_16omp.GFLOPS*(dfn1MacNonFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacNonFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_8mpi_16omp.GFLOPS*(dfn1MacNonFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))







# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Macro nonfused: Probempi vs Nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

# BANDWIDTH

In [18]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_128mpi_1omp.BAND*(dfn1MacNonFusNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacNonFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_128mpi_1omp.BAND*(dfn1MacNonFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_64mpi_2omp.BAND*(dfn1MacNonFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacNonFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_64mpi_2omp.BAND*(dfn1MacNonFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_32mpi_4omp.BAND*(dfn1MacNonFusNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacNonFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_32mpi_4omp.BAND*(dfn1MacNonFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_16mpi_8omp.BAND*(dfn1MacNonFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacNonFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_16mpi_8omp.BAND*(dfn1MacNonFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))



# Case 5
fig.add_trace(go.Scatter(x=(dfn1MacNonFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacNonFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacNonFusNonProb_8mpi_16omp.BAND*(dfn1MacNonFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNonFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacNonFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacNonFusProb_8mpi_16omp.BAND*(dfn1MacNonFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNonFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))


# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=700, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=700
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Fused-Macro (1-Node): probempi vs nonprobempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 

## 5) Newgeo_Macro_Probempi

### Adding Columns

In [19]:
dfn1_macro_newgeo_probempi["N_Nodes"] = 1
dfn1_macro_newgeo_probempi['NPX'] = (dfn1_macro_newgeo_probempi.GLB_X//dfn1_macro_newgeo_probempi.LOC_X)
dfn1_macro_newgeo_probempi['NPY'] = (dfn1_macro_newgeo_probempi.GLB_Y//dfn1_macro_newgeo_probempi.LOC_Y)
dfn1_macro_newgeo_probempi['NPZ'] = (dfn1_macro_newgeo_probempi.GLB_Z//dfn1_macro_newgeo_probempi.LOC_Z)
dfn1_macro_newgeo_probempi['NPT'] = (dfn1_macro_newgeo_probempi.GLB_T//dfn1_macro_newgeo_probempi.LOC_T)
dfn1_macro_newgeo_probempi["Total_proc"] = dfn1_macro_newgeo_probempi["omp_thrd"]* dfn1_macro_newgeo_probempi.MPI_size
# dfn1_macro_newgeo_probempi.columns

### Data Summary

In [20]:
dfn1_macro_newgeo_probempi_summary =dfn1_macro_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_macro_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,69
1,16,8,2,2,2,2,128,70
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [21]:
dfn1MacNewgeoProb_8mpi_16omp = dfn1_macro_newgeo_probempi[(dfn1_macro_newgeo_probempi.omp_thrd == 16) & (dfn1_macro_newgeo_probempi.GLB_T/dfn1_macro_newgeo_probempi.LOC_T == 1) & (dfn1_macro_newgeo_probempi.GLB_X/dfn1_macro_newgeo_probempi.LOC_X == 2) & (dfn1_macro_newgeo_probempi.GLB_Y/dfn1_macro_newgeo_probempi.LOC_Y == 2)& (dfn1_macro_newgeo_probempi.GLB_Z/dfn1_macro_newgeo_probempi.LOC_Z == 2)]
dfn1MacNewgeoProb_16mpi_8omp = dfn1_macro_newgeo_probempi[(dfn1_macro_newgeo_probempi.omp_thrd == 8) & (dfn1_macro_newgeo_probempi.GLB_T/dfn1_macro_newgeo_probempi.LOC_T == 2) & (dfn1_macro_newgeo_probempi.GLB_X/dfn1_macro_newgeo_probempi.LOC_X == 2) & (dfn1_macro_newgeo_probempi.GLB_Y/dfn1_macro_newgeo_probempi.LOC_Y == 2)& (dfn1_macro_newgeo_probempi.GLB_Z/dfn1_macro_newgeo_probempi.LOC_Z == 2)]
dfn1MacNewgeoProb_32mpi_4omp = dfn1_macro_newgeo_probempi[(dfn1_macro_newgeo_probempi.omp_thrd == 4) & (dfn1_macro_newgeo_probempi.GLB_T/dfn1_macro_newgeo_probempi.LOC_T == 4) & (dfn1_macro_newgeo_probempi.GLB_X/dfn1_macro_newgeo_probempi.LOC_X == 2) & (dfn1_macro_newgeo_probempi.GLB_Y/dfn1_macro_newgeo_probempi.LOC_Y == 2)& (dfn1_macro_newgeo_probempi.GLB_Z/dfn1_macro_newgeo_probempi.LOC_Z == 2)]
dfn1MacNewgeoProb_64mpi_2omp = dfn1_macro_newgeo_probempi[(dfn1_macro_newgeo_probempi.omp_thrd == 2) & (dfn1_macro_newgeo_probempi.GLB_T/dfn1_macro_newgeo_probempi.LOC_T == 8) & (dfn1_macro_newgeo_probempi.GLB_X/dfn1_macro_newgeo_probempi.LOC_X == 2) & (dfn1_macro_newgeo_probempi.GLB_Y/dfn1_macro_newgeo_probempi.LOC_Y == 2)& (dfn1_macro_newgeo_probempi.GLB_Z/dfn1_macro_newgeo_probempi.LOC_Z == 2)]
dfn1MacNewgeoProb_128mpi_1omp = dfn1_macro_newgeo_probempi[(dfn1_macro_newgeo_probempi.omp_thrd == 1) & (dfn1_macro_newgeo_probempi.GLB_T/dfn1_macro_newgeo_probempi.LOC_T == 16) & (dfn1_macro_newgeo_probempi.GLB_X/dfn1_macro_newgeo_probempi.LOC_X == 2) & (dfn1_macro_newgeo_probempi.GLB_Y/dfn1_macro_newgeo_probempi.LOC_Y == 2)& (dfn1_macro_newgeo_probempi.GLB_Z/dfn1_macro_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_Macro_NonProbempi

### Adding Columns

In [22]:
dfn1_macro_newgeo_nonprobempi["N_Nodes"] = 1
dfn1_macro_newgeo_nonprobempi['NPX'] = (dfn1_macro_newgeo_nonprobempi.GLB_X//dfn1_macro_newgeo_nonprobempi.LOC_X)
dfn1_macro_newgeo_nonprobempi['NPY'] = (dfn1_macro_newgeo_nonprobempi.GLB_Y//dfn1_macro_newgeo_nonprobempi.LOC_Y)
dfn1_macro_newgeo_nonprobempi['NPZ'] = (dfn1_macro_newgeo_nonprobempi.GLB_Z//dfn1_macro_newgeo_nonprobempi.LOC_Z)
dfn1_macro_newgeo_nonprobempi['NPT'] = (dfn1_macro_newgeo_nonprobempi.GLB_T//dfn1_macro_newgeo_nonprobempi.LOC_T)
dfn1_macro_newgeo_nonprobempi["Total_proc"] = dfn1_macro_newgeo_nonprobempi["omp_thrd"]* dfn1_macro_newgeo_nonprobempi.MPI_size
# dfn1_macro_newgeo_nonprobempi.columns

### Data Summary

In [23]:
dfn1_macro_newgeo_nonprobempi_summary = dfn1_macro_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_macro_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [24]:
dfn1MacNewgeoNonProb_8mpi_16omp = dfn1_macro_newgeo_nonprobempi[(dfn1_macro_newgeo_nonprobempi.omp_thrd == 16) & (dfn1_macro_newgeo_nonprobempi.GLB_T/dfn1_macro_newgeo_nonprobempi.LOC_T == 1) & (dfn1_macro_newgeo_nonprobempi.GLB_X/dfn1_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn1_macro_newgeo_nonprobempi.GLB_Y/dfn1_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_macro_newgeo_nonprobempi.GLB_Z/dfn1_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn1MacNewgeoNonProb_16mpi_8omp = dfn1_macro_newgeo_nonprobempi[(dfn1_macro_newgeo_nonprobempi.omp_thrd == 8) & (dfn1_macro_newgeo_nonprobempi.GLB_T/dfn1_macro_newgeo_nonprobempi.LOC_T == 2) & (dfn1_macro_newgeo_nonprobempi.GLB_X/dfn1_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn1_macro_newgeo_nonprobempi.GLB_Y/dfn1_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_macro_newgeo_nonprobempi.GLB_Z/dfn1_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn1MacNewgeoNonProb_32mpi_4omp = dfn1_macro_newgeo_nonprobempi[(dfn1_macro_newgeo_nonprobempi.omp_thrd == 4) & (dfn1_macro_newgeo_nonprobempi.GLB_T/dfn1_macro_newgeo_nonprobempi.LOC_T == 4) & (dfn1_macro_newgeo_nonprobempi.GLB_X/dfn1_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn1_macro_newgeo_nonprobempi.GLB_Y/dfn1_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_macro_newgeo_nonprobempi.GLB_Z/dfn1_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn1MacNewgeoNonProb_64mpi_2omp = dfn1_macro_newgeo_nonprobempi[(dfn1_macro_newgeo_nonprobempi.omp_thrd == 2) & (dfn1_macro_newgeo_nonprobempi.GLB_T/dfn1_macro_newgeo_nonprobempi.LOC_T == 8) & (dfn1_macro_newgeo_nonprobempi.GLB_X/dfn1_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn1_macro_newgeo_nonprobempi.GLB_Y/dfn1_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_macro_newgeo_nonprobempi.GLB_Z/dfn1_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn1MacNewgeoNonProb_128mpi_1omp = dfn1_macro_newgeo_nonprobempi[(dfn1_macro_newgeo_nonprobempi.omp_thrd == 1) & (dfn1_macro_newgeo_nonprobempi.GLB_T/dfn1_macro_newgeo_nonprobempi.LOC_T == 16) & (dfn1_macro_newgeo_nonprobempi.GLB_X/dfn1_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn1_macro_newgeo_nonprobempi.GLB_Y/dfn1_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_macro_newgeo_nonprobempi.GLB_Z/dfn1_macro_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo Macro: Probempi vs Nonprobempi

In [25]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_128mpi_1omp.GFLOPS*(dfn1MacNewgeoNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_128mpi_1omp.GFLOPS*(dfn1MacNewgeoProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_64mpi_2omp.GFLOPS*(dfn1MacNewgeoNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_64mpi_2omp.GFLOPS*(dfn1MacNewgeoProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_32mpi_4omp.GFLOPS*(dfn1MacNewgeoNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_32mpi_4omp.GFLOPS*(dfn1MacNewgeoProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# Case 4
fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_16mpi_8omp.GFLOPS*(dfn1MacNewgeoNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_16mpi_8omp.GFLOPS*(dfn1MacNewgeoProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))
# Case 5

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_8mpi_16omp.GFLOPS*(dfn1MacNewgeoNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_8mpi_16omp.GFLOPS*(dfn1MacNewgeoProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo Macro: Probempi vs Nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

# BANDWIDTH

In [26]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_128mpi_1omp.BAND*(dfn1MacNewgeoNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_128mpi_1omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_128mpi_1omp.BAND*(dfn1MacNewgeoProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_64mpi_2omp.BAND*(dfn1MacNewgeoNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_64mpi_2omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_64mpi_2omp.BAND*(dfn1MacNewgeoProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_32mpi_4omp.BAND*(dfn1MacNewgeoNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_32mpi_4omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_32mpi_4omp.BAND*(dfn1MacNewgeoProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

# Case 4
fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_16mpi_8omp.BAND*(dfn1MacNewgeoNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_16mpi_8omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_16mpi_8omp.BAND*(dfn1MacNewgeoProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))
# Case 5

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacNewgeoNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacNewgeoNonProb_8mpi_16omp.BAND*(dfn1MacNewgeoNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))

fig.add_trace(go.Scatter(x=(dfn1MacNewgeoProb_8mpi_16omp.Dirc_Data_Mov*(dfn1MacNewgeoProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1MacNewgeoProb_8mpi_16omp.BAND*(dfn1MacNewgeoProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='MacroNewgeoProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))

# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=700, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=700
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Newgeo Macro: Probempi vs Nonprobempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 



# HIPPO AVX CASES

## 1) Fused_AVX_Probmpi

### Adding Columns

In [27]:
dfn1_avx_fused_probmpi["N_Nodes"] = 1
dfn1_avx_fused_probmpi['NPX'] = (dfn1_avx_fused_probmpi.GLB_X//dfn1_avx_fused_probmpi.LOC_X)
dfn1_avx_fused_probmpi['NPY'] = (dfn1_avx_fused_probmpi.GLB_Y//dfn1_avx_fused_probmpi.LOC_Y)
dfn1_avx_fused_probmpi['NPZ'] = (dfn1_avx_fused_probmpi.GLB_Z//dfn1_avx_fused_probmpi.LOC_Z)
dfn1_avx_fused_probmpi['NPT'] = (dfn1_avx_fused_probmpi.GLB_T//dfn1_avx_fused_probmpi.LOC_T)
dfn1_avx_fused_probmpi["Total_proc"] = dfn1_avx_fused_probmpi["omp_thrd"]* dfn1_avx_fused_probmpi.MPI_size
# dfn1_avx_fused_probmpi.columns

### Separating the records that have approx 1 GB data move

In [28]:
# dfn1_avx_fused_probmpi["Total_DataMOV"] = dfn1_avx_fused_probmpi.Dirc_Data_Mov * dfn1_avx_fused_probmpi.MPI_size
# Fused_AVX_Probmpi = dfn1_avx_fused_probmpi[(dfn1_avx_fused_probmpi.Total_DataMOV < 1.277952e+09)]
# Fused_AVX_Probmpi.to_csv('Fused_AVX_Probmpi.csv', index=False)

### Data Summary

In [29]:
dfn1_avx_fused_probmpi_summary = dfn1_avx_fused_probmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_avx_fused_probmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,70
1,16,8,2,2,2,2,128,70
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [30]:
dfn1AvxFusProb_8mpi_16omp = dfn1_avx_fused_probmpi[(dfn1_avx_fused_probmpi.omp_thrd == 16) & (dfn1_avx_fused_probmpi.GLB_T/dfn1_avx_fused_probmpi.LOC_T == 1) & (dfn1_avx_fused_probmpi.GLB_X/dfn1_avx_fused_probmpi.LOC_X == 2) & (dfn1_avx_fused_probmpi.GLB_Y/dfn1_avx_fused_probmpi.LOC_Y == 2)& (dfn1_avx_fused_probmpi.GLB_Z/dfn1_avx_fused_probmpi.LOC_Z == 2)]
dfn1AvxFusProb_16mpi_8omp = dfn1_avx_fused_probmpi[(dfn1_avx_fused_probmpi.omp_thrd == 8) & (dfn1_avx_fused_probmpi.GLB_T/dfn1_avx_fused_probmpi.LOC_T == 2) & (dfn1_avx_fused_probmpi.GLB_X/dfn1_avx_fused_probmpi.LOC_X == 2) & (dfn1_avx_fused_probmpi.GLB_Y/dfn1_avx_fused_probmpi.LOC_Y == 2)& (dfn1_avx_fused_probmpi.GLB_Z/dfn1_avx_fused_probmpi.LOC_Z == 2)]
dfn1AvxFusProb_32mpi_4omp = dfn1_avx_fused_probmpi[(dfn1_avx_fused_probmpi.omp_thrd == 4) & (dfn1_avx_fused_probmpi.GLB_T/dfn1_avx_fused_probmpi.LOC_T == 4) & (dfn1_avx_fused_probmpi.GLB_X/dfn1_avx_fused_probmpi.LOC_X == 2) & (dfn1_avx_fused_probmpi.GLB_Y/dfn1_avx_fused_probmpi.LOC_Y == 2)& (dfn1_avx_fused_probmpi.GLB_Z/dfn1_avx_fused_probmpi.LOC_Z == 2)]
dfn1AvxFusProb_64mpi_2omp = dfn1_avx_fused_probmpi[(dfn1_avx_fused_probmpi.omp_thrd == 2) & (dfn1_avx_fused_probmpi.GLB_T/dfn1_avx_fused_probmpi.LOC_T == 8) & (dfn1_avx_fused_probmpi.GLB_X/dfn1_avx_fused_probmpi.LOC_X == 2) & (dfn1_avx_fused_probmpi.GLB_Y/dfn1_avx_fused_probmpi.LOC_Y == 2)& (dfn1_avx_fused_probmpi.GLB_Z/dfn1_avx_fused_probmpi.LOC_Z == 2)]
dfn1AvxFusProb_128mpi_1omp = dfn1_avx_fused_probmpi[(dfn1_avx_fused_probmpi.omp_thrd == 1) & (dfn1_avx_fused_probmpi.GLB_T/dfn1_avx_fused_probmpi.LOC_T == 16) & (dfn1_avx_fused_probmpi.GLB_X/dfn1_avx_fused_probmpi.LOC_X == 2) & (dfn1_avx_fused_probmpi.GLB_Y/dfn1_avx_fused_probmpi.LOC_Y == 2)& (dfn1_avx_fused_probmpi.GLB_Z/dfn1_avx_fused_probmpi.LOC_Z == 2)]


## 2) Fused_AVX_NonProbmpi

### Adding Columns

In [31]:
dfn1_avx_fused_nonprobmpi["N_Nodes"] = 1
dfn1_avx_fused_nonprobmpi['NPX'] = (dfn1_avx_fused_nonprobmpi.GLB_X//dfn1_avx_fused_nonprobmpi.LOC_X)
dfn1_avx_fused_nonprobmpi['NPY'] = (dfn1_avx_fused_nonprobmpi.GLB_Y//dfn1_avx_fused_nonprobmpi.LOC_Y)
dfn1_avx_fused_nonprobmpi['NPZ'] = (dfn1_avx_fused_nonprobmpi.GLB_Z//dfn1_avx_fused_nonprobmpi.LOC_Z)
dfn1_avx_fused_nonprobmpi['NPT'] = (dfn1_avx_fused_nonprobmpi.GLB_T//dfn1_avx_fused_nonprobmpi.LOC_T)
dfn1_avx_fused_nonprobmpi["Total_proc"] = dfn1_avx_fused_nonprobmpi["omp_thrd"]* dfn1_avx_fused_nonprobmpi.MPI_size

# dfn1_avx_fused_nonprobmpi.columns

### Separating the records that have approx 1 GB data move

In [32]:
# dfn1_avx_fused_nonprobmpi["Total_DataMOV"] = dfn1_avx_fused_nonprobmpi.Dirc_Data_Mov * dfn1_avx_fused_nonprobmpi.MPI_size
# Fused_AVX_NonProbmpi = dfn1_avx_fused_nonprobmpi[(dfn1_avx_fused_nonprobmpi.Total_DataMOV < 1.277952e+09)]
# Fused_AVX_NonProbmpi.to_csv('Fused_AVX_NonProbmpi.csv', index=False)

### Data Summary

In [33]:
dfn1_avx_fused_nonprobmpi_summary = dfn1_avx_fused_nonprobmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_avx_fused_nonprobmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [34]:
dfn1AvxFusNonProb_8mpi_16omp = dfn1_avx_fused_nonprobmpi[(dfn1_avx_fused_nonprobmpi.omp_thrd == 16) & (dfn1_avx_fused_nonprobmpi.GLB_T/dfn1_avx_fused_nonprobmpi.LOC_T == 1) & (dfn1_avx_fused_nonprobmpi.GLB_X/dfn1_avx_fused_nonprobmpi.LOC_X == 2) & (dfn1_avx_fused_nonprobmpi.GLB_Y/dfn1_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn1_avx_fused_nonprobmpi.GLB_Z/dfn1_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn1AvxFusNonProb_16mpi_8omp = dfn1_avx_fused_nonprobmpi[(dfn1_avx_fused_nonprobmpi.omp_thrd == 8) & (dfn1_avx_fused_nonprobmpi.GLB_T/dfn1_avx_fused_nonprobmpi.LOC_T == 2) & (dfn1_avx_fused_nonprobmpi.GLB_X/dfn1_avx_fused_nonprobmpi.LOC_X == 2) & (dfn1_avx_fused_nonprobmpi.GLB_Y/dfn1_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn1_avx_fused_nonprobmpi.GLB_Z/dfn1_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn1AvxFusNonProb_32mpi_4omp = dfn1_avx_fused_nonprobmpi[(dfn1_avx_fused_nonprobmpi.omp_thrd == 4) & (dfn1_avx_fused_nonprobmpi.GLB_T/dfn1_avx_fused_nonprobmpi.LOC_T == 4) & (dfn1_avx_fused_nonprobmpi.GLB_X/dfn1_avx_fused_nonprobmpi.LOC_X == 2) & (dfn1_avx_fused_nonprobmpi.GLB_Y/dfn1_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn1_avx_fused_nonprobmpi.GLB_Z/dfn1_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn1AvxFusNonProb_64mpi_2omp = dfn1_avx_fused_nonprobmpi[(dfn1_avx_fused_nonprobmpi.omp_thrd == 2) & (dfn1_avx_fused_nonprobmpi.GLB_T/dfn1_avx_fused_nonprobmpi.LOC_T == 8) & (dfn1_avx_fused_nonprobmpi.GLB_X/dfn1_avx_fused_nonprobmpi.LOC_X == 2) & (dfn1_avx_fused_nonprobmpi.GLB_Y/dfn1_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn1_avx_fused_nonprobmpi.GLB_Z/dfn1_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn1AvxFusNonProb_128mpi_1omp = dfn1_avx_fused_nonprobmpi[(dfn1_avx_fused_nonprobmpi.omp_thrd == 1) & (dfn1_avx_fused_nonprobmpi.GLB_T/dfn1_avx_fused_nonprobmpi.LOC_T == 16) & (dfn1_avx_fused_nonprobmpi.GLB_X/dfn1_avx_fused_nonprobmpi.LOC_X == 2) & (dfn1_avx_fused_nonprobmpi.GLB_Y/dfn1_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn1_avx_fused_nonprobmpi.GLB_Z/dfn1_avx_fused_nonprobmpi.LOC_Z == 2)]


## GFLOPS - AVX-Fused: Probmpi vs Non-Probempi

In [35]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_128mpi_1omp.GFLOPS*(dfn1AvxFusNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1AvxFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1AvxFusProb_128mpi_1omp.GFLOPS*(dfn1AvxFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_64mpi_2omp.GFLOPS*(dfn1AvxFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1AvxFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1AvxFusProb_64mpi_2omp.GFLOPS*(dfn1AvxFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='dfn1AvxFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_32mpi_4omp.GFLOPS*(dfn1AvxFusNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1AvxFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1AvxFusProb_32mpi_4omp.GFLOPS*(dfn1AvxFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4

fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_16mpi_8omp.GFLOPS*(dfn1AvxFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1AvxFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1AvxFusProb_16mpi_8omp.GFLOPS*(dfn1AvxFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))


# Case 5

fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_8mpi_16omp.GFLOPS*(dfn1AvxFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1AvxFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1AvxFusProb_8mpi_16omp.GFLOPS*(dfn1AvxFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Fused-AVX: Probmpi vs Non-Probempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

# BANDWIDTH

In [36]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_128mpi_1omp.BAND*(dfn1AvxFusNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1AvxFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1AvxFusProb_128mpi_1omp.BAND*(dfn1AvxFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_64mpi_2omp.BAND*(dfn1AvxFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1AvxFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1AvxFusProb_64mpi_2omp.BAND*(dfn1AvxFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='dfn1AvxFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_32mpi_4omp.BAND*(dfn1AvxFusNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1AvxFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1AvxFusProb_32mpi_4omp.BAND*(dfn1AvxFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4

fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_16mpi_8omp.BAND*(dfn1AvxFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1AvxFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1AvxFusProb_16mpi_8omp.BAND*(dfn1AvxFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))


# Case 5

fig.add_trace(go.Scatter(x=(dfn1AvxFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1AvxFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1AvxFusNonProb_8mpi_16omp.BAND*(dfn1AvxFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1AvxFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1AvxFusProb_8mpi_16omp.BAND*(dfn1AvxFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))




# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=700, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 80M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=700
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Fused-AVX: Probmpi vs Non-Probempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 


## 3) NonFused_AVX_Probempi

### Adding Columns

In [37]:
dfn1_avx_nonfused_probempi["N_Nodes"] = 1
dfn1_avx_nonfused_probempi['NPX'] = (dfn1_avx_nonfused_probempi.GLB_X//dfn1_avx_nonfused_probempi.LOC_X)
dfn1_avx_nonfused_probempi['NPY'] = (dfn1_avx_nonfused_probempi.GLB_Y//dfn1_avx_nonfused_probempi.LOC_Y)
dfn1_avx_nonfused_probempi['NPZ'] = (dfn1_avx_nonfused_probempi.GLB_Z//dfn1_avx_nonfused_probempi.LOC_Z)
dfn1_avx_nonfused_probempi['NPT'] = (dfn1_avx_nonfused_probempi.GLB_T//dfn1_avx_nonfused_probempi.LOC_T)
dfn1_avx_nonfused_probempi["Total_proc"] = dfn1_avx_nonfused_probempi["omp_thrd"]* dfn1_avx_nonfused_probempi.MPI_size
# dfn1_avx_nonfused_probempi.columns

### Data Summary

In [38]:
dfn1_avx_nonfused_probempi_summary = dfn1_avx_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_avx_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,70
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,6


### Cases Setup

In [39]:
dfn1AvxNonFusProb_8mpi_16omp = dfn1_avx_nonfused_probempi[(dfn1_avx_nonfused_probempi.omp_thrd == 16) & (dfn1_avx_nonfused_probempi.GLB_T/dfn1_avx_nonfused_probempi.LOC_T == 1) & (dfn1_avx_nonfused_probempi.GLB_X/dfn1_avx_nonfused_probempi.LOC_X == 2) & (dfn1_avx_nonfused_probempi.GLB_Y/dfn1_avx_nonfused_probempi.LOC_Y == 2)& (dfn1_avx_nonfused_probempi.GLB_Z/dfn1_avx_nonfused_probempi.LOC_Z == 2)]
dfn1AvxNonFusProb_16mpi_8omp = dfn1_avx_nonfused_probempi[(dfn1_avx_nonfused_probempi.omp_thrd == 8) & (dfn1_avx_nonfused_probempi.GLB_T/dfn1_avx_nonfused_probempi.LOC_T == 2) & (dfn1_avx_nonfused_probempi.GLB_X/dfn1_avx_nonfused_probempi.LOC_X == 2) & (dfn1_avx_nonfused_probempi.GLB_Y/dfn1_avx_nonfused_probempi.LOC_Y == 2)& (dfn1_avx_nonfused_probempi.GLB_Z/dfn1_avx_nonfused_probempi.LOC_Z == 2)]
dfn1AvxNonFusProb_32mpi_4omp = dfn1_avx_nonfused_probempi[(dfn1_avx_nonfused_probempi.omp_thrd == 4) & (dfn1_avx_nonfused_probempi.GLB_T/dfn1_avx_nonfused_probempi.LOC_T == 4) & (dfn1_avx_nonfused_probempi.GLB_X/dfn1_avx_nonfused_probempi.LOC_X == 2) & (dfn1_avx_nonfused_probempi.GLB_Y/dfn1_avx_nonfused_probempi.LOC_Y == 2)& (dfn1_avx_nonfused_probempi.GLB_Z/dfn1_avx_nonfused_probempi.LOC_Z == 2)]
dfn1AvxNonFusProb_64mpi_2omp = dfn1_avx_nonfused_probempi[(dfn1_avx_nonfused_probempi.omp_thrd == 2) & (dfn1_avx_nonfused_probempi.GLB_T/dfn1_avx_nonfused_probempi.LOC_T == 8) & (dfn1_avx_nonfused_probempi.GLB_X/dfn1_avx_nonfused_probempi.LOC_X == 2) & (dfn1_avx_nonfused_probempi.GLB_Y/dfn1_avx_nonfused_probempi.LOC_Y == 2)& (dfn1_avx_nonfused_probempi.GLB_Z/dfn1_avx_nonfused_probempi.LOC_Z == 2)]
dfn1AvxNonFusProb_128mpi_1omp = dfn1_avx_nonfused_probempi[(dfn1_avx_nonfused_probempi.omp_thrd == 1) & (dfn1_avx_nonfused_probempi.GLB_T/dfn1_avx_nonfused_probempi.LOC_T == 16) & (dfn1_avx_nonfused_probempi.GLB_X/dfn1_avx_nonfused_probempi.LOC_X == 2) & (dfn1_avx_nonfused_probempi.GLB_Y/dfn1_avx_nonfused_probempi.LOC_Y == 2)& (dfn1_avx_nonfused_probempi.GLB_Z/dfn1_avx_nonfused_probempi.LOC_Z == 2)]


## 4) NonFused_AVX_NonProbempi

### Adding Columns

In [40]:
dfn1_avx_nonfused_nonprobempi["N_Nodes"] = 1
dfn1_avx_nonfused_nonprobempi['NPX'] = (dfn1_avx_nonfused_nonprobempi.GLB_X//dfn1_avx_nonfused_nonprobempi.LOC_X)
dfn1_avx_nonfused_nonprobempi['NPY'] = (dfn1_avx_nonfused_nonprobempi.GLB_Y//dfn1_avx_nonfused_nonprobempi.LOC_Y)
dfn1_avx_nonfused_nonprobempi['NPZ'] = (dfn1_avx_nonfused_nonprobempi.GLB_Z//dfn1_avx_nonfused_nonprobempi.LOC_Z)
dfn1_avx_nonfused_nonprobempi['NPT'] = (dfn1_avx_nonfused_nonprobempi.GLB_T//dfn1_avx_nonfused_nonprobempi.LOC_T)
dfn1_avx_nonfused_nonprobempi["Total_proc"] = dfn1_avx_nonfused_nonprobempi["omp_thrd"]* dfn1_avx_nonfused_nonprobempi.MPI_size
# dfn1_avx_nonfused_nonprobempi.columns

### Data Summary

In [41]:
dfn1_avx_nonfused_nonprobempi_summary = dfn1_avx_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_avx_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [42]:
dfn1AvxNonFusNonProb_8mpi_16omp = dfn1_avx_nonfused_nonprobempi[(dfn1_avx_nonfused_nonprobempi.omp_thrd == 16) & (dfn1_avx_nonfused_nonprobempi.GLB_T/dfn1_avx_nonfused_nonprobempi.LOC_T == 1) & (dfn1_avx_nonfused_nonprobempi.GLB_X/dfn1_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn1_avx_nonfused_nonprobempi.GLB_Y/dfn1_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_avx_nonfused_nonprobempi.GLB_Z/dfn1_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn1AvxNonFusNonProb_16mpi_8omp = dfn1_avx_nonfused_nonprobempi[(dfn1_avx_nonfused_nonprobempi.omp_thrd == 8) & (dfn1_avx_nonfused_nonprobempi.GLB_T/dfn1_avx_nonfused_nonprobempi.LOC_T == 2) & (dfn1_avx_nonfused_nonprobempi.GLB_X/dfn1_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn1_avx_nonfused_nonprobempi.GLB_Y/dfn1_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_avx_nonfused_nonprobempi.GLB_Z/dfn1_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn1AvxNonFusNonProb_32mpi_4omp = dfn1_avx_nonfused_nonprobempi[(dfn1_avx_nonfused_nonprobempi.omp_thrd == 4) & (dfn1_avx_nonfused_nonprobempi.GLB_T/dfn1_avx_nonfused_nonprobempi.LOC_T == 4) & (dfn1_avx_nonfused_nonprobempi.GLB_X/dfn1_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn1_avx_nonfused_nonprobempi.GLB_Y/dfn1_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_avx_nonfused_nonprobempi.GLB_Z/dfn1_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn1AvxNonFusNonProb_64mpi_2omp = dfn1_avx_nonfused_nonprobempi[(dfn1_avx_nonfused_nonprobempi.omp_thrd == 2) & (dfn1_avx_nonfused_nonprobempi.GLB_T/dfn1_avx_nonfused_nonprobempi.LOC_T == 8) & (dfn1_avx_nonfused_nonprobempi.GLB_X/dfn1_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn1_avx_nonfused_nonprobempi.GLB_Y/dfn1_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_avx_nonfused_nonprobempi.GLB_Z/dfn1_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn1AvxNonFusNonProb_128mpi_1omp = dfn1_avx_nonfused_nonprobempi[(dfn1_avx_nonfused_nonprobempi.omp_thrd == 1) & (dfn1_avx_nonfused_nonprobempi.GLB_T/dfn1_avx_nonfused_nonprobempi.LOC_T == 16) & (dfn1_avx_nonfused_nonprobempi.GLB_X/dfn1_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn1_avx_nonfused_nonprobempi.GLB_Y/dfn1_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_avx_nonfused_nonprobempi.GLB_Z/dfn1_avx_nonfused_nonprobempi.LOC_Z == 2)]


# GFLOPS - NonFused-AVX: Probmpi vs Non-Probempi

In [43]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1AvxNonFusNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1AvxNonFusNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1AvxNonFusNonProb_128mpi_1omp.GFLOPS*(dfn1AvxNonFusNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1AvxNonFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1AvxNonFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1AvxNonFusProb_128mpi_1omp.GFLOPS*(dfn1AvxNonFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1AvxNonFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1AvxNonFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1AvxNonFusNonProb_64mpi_2omp.GFLOPS*(dfn1AvxNonFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1AvxNonFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1AvxNonFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1AvxNonFusProb_64mpi_2omp.GFLOPS*(dfn1AvxNonFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1AvxNonFusNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1AvxNonFusNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1AvxNonFusNonProb_32mpi_4omp.GFLOPS*(dfn1AvxNonFusNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxNonFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1AvxNonFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1AvxNonFusProb_32mpi_4omp.GFLOPS*(dfn1AvxNonFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4

fig.add_trace(go.Scatter(x=(dfn1AvxNonFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1AvxNonFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1AvxNonFusNonProb_16mpi_8omp.GFLOPS*(dfn1AvxNonFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxNonFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1AvxNonFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1AvxNonFusProb_16mpi_8omp.GFLOPS*(dfn1AvxNonFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))


# Case 5

fig.add_trace(go.Scatter(x=(dfn1AvxNonFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1AvxNonFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1AvxNonFusNonProb_8mpi_16omp.GFLOPS*(dfn1AvxNonFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxNonFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1AvxNonFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1AvxNonFusProb_8mpi_16omp.GFLOPS*(dfn1AvxNonFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-AVX: Probmpi vs Non-Probempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 5) Newgeo_AVX_Probempi

### Adding Columns

In [44]:
dfn1_avx_newgeo_probempi["N_Nodes"] = 1
dfn1_avx_newgeo_probempi['NPX'] = (dfn1_avx_newgeo_probempi.GLB_X//dfn1_avx_newgeo_probempi.LOC_X)
dfn1_avx_newgeo_probempi['NPY'] = (dfn1_avx_newgeo_probempi.GLB_Y//dfn1_avx_newgeo_probempi.LOC_Y)
dfn1_avx_newgeo_probempi['NPZ'] = (dfn1_avx_newgeo_probempi.GLB_Z//dfn1_avx_newgeo_probempi.LOC_Z)
dfn1_avx_newgeo_probempi['NPT'] = (dfn1_avx_newgeo_probempi.GLB_T//dfn1_avx_newgeo_probempi.LOC_T)
dfn1_avx_newgeo_probempi["Total_proc"] = dfn1_avx_newgeo_probempi["omp_thrd"]* dfn1_avx_newgeo_probempi.MPI_size
# dfn1_avx_newgeo_probempi.columns

### Data Summary

In [45]:
dfn1_avx_newgeo_probempi_summary = dfn1_avx_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_avx_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,70
1,16,8,2,2,2,2,128,70
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [46]:
dfn1AvxNewgeoProb_8mpi_16omp = dfn1_avx_newgeo_probempi[(dfn1_avx_newgeo_probempi.omp_thrd == 16) & (dfn1_avx_newgeo_probempi.GLB_T/dfn1_avx_newgeo_probempi.LOC_T == 1) & (dfn1_avx_newgeo_probempi.GLB_X/dfn1_avx_newgeo_probempi.LOC_X == 2) & (dfn1_avx_newgeo_probempi.GLB_Y/dfn1_avx_newgeo_probempi.LOC_Y == 2)& (dfn1_avx_newgeo_probempi.GLB_Z/dfn1_avx_newgeo_probempi.LOC_Z == 2)]
dfn1AvxNewgeoProb_16mpi_8omp = dfn1_avx_newgeo_probempi[(dfn1_avx_newgeo_probempi.omp_thrd == 8) & (dfn1_avx_newgeo_probempi.GLB_T/dfn1_avx_newgeo_probempi.LOC_T == 2) & (dfn1_avx_newgeo_probempi.GLB_X/dfn1_avx_newgeo_probempi.LOC_X == 2) & (dfn1_avx_newgeo_probempi.GLB_Y/dfn1_avx_newgeo_probempi.LOC_Y == 2)& (dfn1_avx_newgeo_probempi.GLB_Z/dfn1_avx_newgeo_probempi.LOC_Z == 2)]
dfn1AvxNewgeoProb_32mpi_4omp = dfn1_avx_newgeo_probempi[(dfn1_avx_newgeo_probempi.omp_thrd == 4) & (dfn1_avx_newgeo_probempi.GLB_T/dfn1_avx_newgeo_probempi.LOC_T == 4) & (dfn1_avx_newgeo_probempi.GLB_X/dfn1_avx_newgeo_probempi.LOC_X == 2) & (dfn1_avx_newgeo_probempi.GLB_Y/dfn1_avx_newgeo_probempi.LOC_Y == 2)& (dfn1_avx_newgeo_probempi.GLB_Z/dfn1_avx_newgeo_probempi.LOC_Z == 2)]
dfn1AvxNewgeoProb_64mpi_2omp = dfn1_avx_newgeo_probempi[(dfn1_avx_newgeo_probempi.omp_thrd == 2) & (dfn1_avx_newgeo_probempi.GLB_T/dfn1_avx_newgeo_probempi.LOC_T == 8) & (dfn1_avx_newgeo_probempi.GLB_X/dfn1_avx_newgeo_probempi.LOC_X == 2) & (dfn1_avx_newgeo_probempi.GLB_Y/dfn1_avx_newgeo_probempi.LOC_Y == 2)& (dfn1_avx_newgeo_probempi.GLB_Z/dfn1_avx_newgeo_probempi.LOC_Z == 2)]
dfn1AvxNewgeoProb_128mpi_1omp = dfn1_avx_newgeo_probempi[(dfn1_avx_newgeo_probempi.omp_thrd == 1) & (dfn1_avx_newgeo_probempi.GLB_T/dfn1_avx_newgeo_probempi.LOC_T == 16) & (dfn1_avx_newgeo_probempi.GLB_X/dfn1_avx_newgeo_probempi.LOC_X == 2) & (dfn1_avx_newgeo_probempi.GLB_Y/dfn1_avx_newgeo_probempi.LOC_Y == 2)& (dfn1_avx_newgeo_probempi.GLB_Z/dfn1_avx_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_AVX_NonProbempi

### Adding Columns

In [47]:
dfn1_avx_newgeo_nonprobempi["N_Nodes"] = 1
dfn1_avx_newgeo_nonprobempi['NPX'] = (dfn1_avx_newgeo_nonprobempi.GLB_X//dfn1_avx_newgeo_nonprobempi.LOC_X)
dfn1_avx_newgeo_nonprobempi['NPY'] = (dfn1_avx_newgeo_nonprobempi.GLB_Y//dfn1_avx_newgeo_nonprobempi.LOC_Y)
dfn1_avx_newgeo_nonprobempi['NPZ'] = (dfn1_avx_newgeo_nonprobempi.GLB_Z//dfn1_avx_newgeo_nonprobempi.LOC_Z)
dfn1_avx_newgeo_nonprobempi['NPT'] = (dfn1_avx_newgeo_nonprobempi.GLB_T//dfn1_avx_newgeo_nonprobempi.LOC_T)
dfn1_avx_newgeo_nonprobempi["Total_proc"] = dfn1_avx_newgeo_nonprobempi["omp_thrd"]* dfn1_avx_newgeo_nonprobempi.MPI_size
# dfn1_avx_newgeo_nonprobempi.columns

### Data Summary

In [48]:
dfn1_avx_newgeo_nonprobempi_summary = dfn1_avx_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_avx_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,16
4,128,1,16,2,2,2,128,7


### Cases Setup

In [49]:
dfn1AvxNewgeoNonProb_8mpi_16omp = dfn1_avx_newgeo_nonprobempi[(dfn1_avx_newgeo_nonprobempi.omp_thrd == 16) & (dfn1_avx_newgeo_nonprobempi.GLB_T/dfn1_avx_newgeo_nonprobempi.LOC_T == 1) & (dfn1_avx_newgeo_nonprobempi.GLB_X/dfn1_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn1_avx_newgeo_nonprobempi.GLB_Y/dfn1_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_avx_newgeo_nonprobempi.GLB_Z/dfn1_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn1AvxNewgeoNonProb_16mpi_8omp = dfn1_avx_newgeo_nonprobempi[(dfn1_avx_newgeo_nonprobempi.omp_thrd == 8) & (dfn1_avx_newgeo_nonprobempi.GLB_T/dfn1_avx_newgeo_nonprobempi.LOC_T == 2) & (dfn1_avx_newgeo_nonprobempi.GLB_X/dfn1_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn1_avx_newgeo_nonprobempi.GLB_Y/dfn1_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_avx_newgeo_nonprobempi.GLB_Z/dfn1_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn1AvxNewgeoNonProb_32mpi_4omp = dfn1_avx_newgeo_nonprobempi[(dfn1_avx_newgeo_nonprobempi.omp_thrd == 4) & (dfn1_avx_newgeo_nonprobempi.GLB_T/dfn1_avx_newgeo_nonprobempi.LOC_T == 4) & (dfn1_avx_newgeo_nonprobempi.GLB_X/dfn1_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn1_avx_newgeo_nonprobempi.GLB_Y/dfn1_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_avx_newgeo_nonprobempi.GLB_Z/dfn1_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn1AvxNewgeoNonProb_64mpi_2omp = dfn1_avx_newgeo_nonprobempi[(dfn1_avx_newgeo_nonprobempi.omp_thrd == 2) & (dfn1_avx_newgeo_nonprobempi.GLB_T/dfn1_avx_newgeo_nonprobempi.LOC_T == 8) & (dfn1_avx_newgeo_nonprobempi.GLB_X/dfn1_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn1_avx_newgeo_nonprobempi.GLB_Y/dfn1_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_avx_newgeo_nonprobempi.GLB_Z/dfn1_avx_newgeo_nonprobempi.LOC_Z == 2)]
dfn1AvxNewgeoNonProb_128mpi_1omp = dfn1_avx_newgeo_nonprobempi[(dfn1_avx_newgeo_nonprobempi.omp_thrd == 1) & (dfn1_avx_newgeo_nonprobempi.GLB_T/dfn1_avx_newgeo_nonprobempi.LOC_T == 16) & (dfn1_avx_newgeo_nonprobempi.GLB_X/dfn1_avx_newgeo_nonprobempi.LOC_X == 2) & (dfn1_avx_newgeo_nonprobempi.GLB_Y/dfn1_avx_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_avx_newgeo_nonprobempi.GLB_Z/dfn1_avx_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo-AVX: Probmpi vs Non-Probempi

In [50]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1AvxNewgeoNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1AvxNewgeoNonProb_128mpi_1omp.GFLOPS*(dfn1AvxNewgeoNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoProb_128mpi_1omp.Dirc_Data_Mov*(dfn1AvxNewgeoProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1AvxNewgeoProb_128mpi_1omp.GFLOPS*(dfn1AvxNewgeoProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1AvxNewgeoNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1AvxNewgeoNonProb_64mpi_2omp.GFLOPS*(dfn1AvxNewgeoNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoProb_64mpi_2omp.Dirc_Data_Mov*(dfn1AvxNewgeoProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1AvxNewgeoProb_64mpi_2omp.GFLOPS*(dfn1AvxNewgeoProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1AvxNewgeoNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1AvxNewgeoNonProb_32mpi_4omp.GFLOPS*(dfn1AvxNewgeoNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='dfn1AvxNewgeoNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoProb_32mpi_4omp.Dirc_Data_Mov*(dfn1AvxNewgeoProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1AvxNewgeoProb_32mpi_4omp.GFLOPS*(dfn1AvxNewgeoProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4

fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1AvxNewgeoNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1AvxNewgeoNonProb_16mpi_8omp.GFLOPS*(dfn1AvxNewgeoNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoProb_16mpi_8omp.Dirc_Data_Mov*(dfn1AvxNewgeoProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1AvxNewgeoProb_16mpi_8omp.GFLOPS*(dfn1AvxNewgeoProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))


# Case 5

fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1AvxNewgeoNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1AvxNewgeoNonProb_8mpi_16omp.GFLOPS*(dfn1AvxNewgeoNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))


fig.add_trace(go.Scatter(x=(dfn1AvxNewgeoProb_8mpi_16omp.Dirc_Data_Mov*(dfn1AvxNewgeoProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1AvxNewgeoProb_8mpi_16omp.GFLOPS*(dfn1AvxNewgeoProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='AvxNewgeoProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo-AVX: Probmpi vs Non-Probempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

# HIPPO VECT CASES

## 1) Fused_VECT_Probmpi 

### Adding Columns

In [51]:
dfn1_vect_fused_probmpi["N_Nodes"] = 1
dfn1_vect_fused_probmpi['NPX'] = (dfn1_vect_fused_probmpi.GLB_X//dfn1_vect_fused_probmpi.LOC_X)
dfn1_vect_fused_probmpi['NPY'] = (dfn1_vect_fused_probmpi.GLB_Y//dfn1_vect_fused_probmpi.LOC_Y)
dfn1_vect_fused_probmpi['NPZ'] = (dfn1_vect_fused_probmpi.GLB_Z//dfn1_vect_fused_probmpi.LOC_Z)
dfn1_vect_fused_probmpi['NPT'] = (dfn1_vect_fused_probmpi.GLB_T//dfn1_vect_fused_probmpi.LOC_T)
dfn1_vect_fused_probmpi["Total_proc"] = dfn1_vect_fused_probmpi["omp_thrd"]* dfn1_vect_fused_probmpi.MPI_size
# dfn1_vect_fused_probmpi.columns

### Data Summary

In [52]:
dfn1_vect_fused_probmpi_summary = dfn1_vect_fused_probmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_vect_fused_probmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [53]:
dfn1VectFusProb_8mpi_16omp = dfn1_vect_fused_probmpi[(dfn1_vect_fused_probmpi.omp_thrd == 16) & (dfn1_vect_fused_probmpi.GLB_T/dfn1_vect_fused_probmpi.LOC_T == 1) & (dfn1_vect_fused_probmpi.GLB_X/dfn1_vect_fused_probmpi.LOC_X == 2) & (dfn1_vect_fused_probmpi.GLB_Y/dfn1_vect_fused_probmpi.LOC_Y == 2)& (dfn1_vect_fused_probmpi.GLB_Z/dfn1_vect_fused_probmpi.LOC_Z == 2)]
dfn1VectFusProb_16mpi_8omp = dfn1_vect_fused_probmpi[(dfn1_vect_fused_probmpi.omp_thrd == 8) & (dfn1_vect_fused_probmpi.GLB_T/dfn1_vect_fused_probmpi.LOC_T == 2) & (dfn1_vect_fused_probmpi.GLB_X/dfn1_vect_fused_probmpi.LOC_X == 2) & (dfn1_vect_fused_probmpi.GLB_Y/dfn1_vect_fused_probmpi.LOC_Y == 2)& (dfn1_vect_fused_probmpi.GLB_Z/dfn1_vect_fused_probmpi.LOC_Z == 2)]
dfn1VectFusProb_32mpi_4omp = dfn1_vect_fused_probmpi[(dfn1_vect_fused_probmpi.omp_thrd == 4) & (dfn1_vect_fused_probmpi.GLB_T/dfn1_vect_fused_probmpi.LOC_T == 4) & (dfn1_vect_fused_probmpi.GLB_X/dfn1_vect_fused_probmpi.LOC_X == 2) & (dfn1_vect_fused_probmpi.GLB_Y/dfn1_vect_fused_probmpi.LOC_Y == 2)& (dfn1_vect_fused_probmpi.GLB_Z/dfn1_vect_fused_probmpi.LOC_Z == 2)]
dfn1VectFusProb_64mpi_2omp = dfn1_vect_fused_probmpi[(dfn1_vect_fused_probmpi.omp_thrd == 2) & (dfn1_vect_fused_probmpi.GLB_T/dfn1_vect_fused_probmpi.LOC_T == 8) & (dfn1_vect_fused_probmpi.GLB_X/dfn1_vect_fused_probmpi.LOC_X == 2) & (dfn1_vect_fused_probmpi.GLB_Y/dfn1_vect_fused_probmpi.LOC_Y == 2)& (dfn1_vect_fused_probmpi.GLB_Z/dfn1_vect_fused_probmpi.LOC_Z == 2)]
dfn1VectFusProb_128mpi_1omp = dfn1_vect_fused_probmpi[(dfn1_vect_fused_probmpi.omp_thrd == 1) & (dfn1_vect_fused_probmpi.GLB_T/dfn1_vect_fused_probmpi.LOC_T == 16) & (dfn1_vect_fused_probmpi.GLB_X/dfn1_vect_fused_probmpi.LOC_X == 2) & (dfn1_vect_fused_probmpi.GLB_Y/dfn1_vect_fused_probmpi.LOC_Y == 2)& (dfn1_vect_fused_probmpi.GLB_Z/dfn1_vect_fused_probmpi.LOC_Z == 2)]


## 2) Fused_VECT_NonProbmpi 

### Adding Columns

In [54]:
dfn1_vect_fused_nonprobmpi["N_Nodes"] = 1
dfn1_vect_fused_nonprobmpi['NPX'] = (dfn1_vect_fused_nonprobmpi.GLB_X//dfn1_vect_fused_nonprobmpi.LOC_X)
dfn1_vect_fused_nonprobmpi['NPY'] = (dfn1_vect_fused_nonprobmpi.GLB_Y//dfn1_vect_fused_nonprobmpi.LOC_Y)
dfn1_vect_fused_nonprobmpi['NPZ'] = (dfn1_vect_fused_nonprobmpi.GLB_Z//dfn1_vect_fused_nonprobmpi.LOC_Z)
dfn1_vect_fused_nonprobmpi['NPT'] = (dfn1_vect_fused_nonprobmpi.GLB_T//dfn1_vect_fused_nonprobmpi.LOC_T)
dfn1_vect_fused_nonprobmpi["Total_proc"] = dfn1_vect_fused_nonprobmpi["omp_thrd"]* dfn1_vect_fused_nonprobmpi.MPI_size
# dfn1_vect_fused_nonprobmpi.columns

### Data Summary

In [55]:
dfn1_vect_fused_nonprobmpi_summary = dfn1_vect_fused_nonprobmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_vect_fused_nonprobmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,70
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [56]:
dfn1VectFusNonProb_8mpi_16omp = dfn1_vect_fused_nonprobmpi[(dfn1_vect_fused_nonprobmpi.omp_thrd == 16) & (dfn1_vect_fused_nonprobmpi.GLB_T/dfn1_vect_fused_nonprobmpi.LOC_T == 1) & (dfn1_vect_fused_nonprobmpi.GLB_X/dfn1_vect_fused_nonprobmpi.LOC_X == 2) & (dfn1_vect_fused_nonprobmpi.GLB_Y/dfn1_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn1_vect_fused_nonprobmpi.GLB_Z/dfn1_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn1VectFusNonProb_16mpi_8omp = dfn1_vect_fused_nonprobmpi[(dfn1_vect_fused_nonprobmpi.omp_thrd == 8) & (dfn1_vect_fused_nonprobmpi.GLB_T/dfn1_vect_fused_nonprobmpi.LOC_T == 2) & (dfn1_vect_fused_nonprobmpi.GLB_X/dfn1_vect_fused_nonprobmpi.LOC_X == 2) & (dfn1_vect_fused_nonprobmpi.GLB_Y/dfn1_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn1_vect_fused_nonprobmpi.GLB_Z/dfn1_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn1VectFusNonProb_32mpi_4omp = dfn1_vect_fused_nonprobmpi[(dfn1_vect_fused_nonprobmpi.omp_thrd == 4) & (dfn1_vect_fused_nonprobmpi.GLB_T/dfn1_vect_fused_nonprobmpi.LOC_T == 4) & (dfn1_vect_fused_nonprobmpi.GLB_X/dfn1_vect_fused_nonprobmpi.LOC_X == 2) & (dfn1_vect_fused_nonprobmpi.GLB_Y/dfn1_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn1_vect_fused_nonprobmpi.GLB_Z/dfn1_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn1VectFusNonProb_64mpi_2omp = dfn1_vect_fused_nonprobmpi[(dfn1_vect_fused_nonprobmpi.omp_thrd == 2) & (dfn1_vect_fused_nonprobmpi.GLB_T/dfn1_vect_fused_nonprobmpi.LOC_T == 8) & (dfn1_vect_fused_nonprobmpi.GLB_X/dfn1_vect_fused_nonprobmpi.LOC_X == 2) & (dfn1_vect_fused_nonprobmpi.GLB_Y/dfn1_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn1_vect_fused_nonprobmpi.GLB_Z/dfn1_vect_fused_nonprobmpi.LOC_Z == 2)]
dfn1VectFusNonProb_128mpi_1omp = dfn1_vect_fused_nonprobmpi[(dfn1_vect_fused_nonprobmpi.omp_thrd == 1) & (dfn1_vect_fused_nonprobmpi.GLB_T/dfn1_vect_fused_nonprobmpi.LOC_T == 16) & (dfn1_vect_fused_nonprobmpi.GLB_X/dfn1_vect_fused_nonprobmpi.LOC_X == 2) & (dfn1_vect_fused_nonprobmpi.GLB_Y/dfn1_vect_fused_nonprobmpi.LOC_Y == 2)& (dfn1_vect_fused_nonprobmpi.GLB_Z/dfn1_vect_fused_nonprobmpi.LOC_Z == 2)]


## GFLOPS - Fused VECT: probempi vs nonprobempi

In [57]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1VectFusNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1VectFusNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1VectFusNonProb_128mpi_1omp.GFLOPS*(dfn1VectFusNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1VectFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1VectFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1VectFusProb_128mpi_1omp.GFLOPS*(dfn1VectFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1VectFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1VectFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1VectFusNonProb_64mpi_2omp.GFLOPS*(dfn1VectFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1VectFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1VectFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1VectFusProb_64mpi_2omp.GFLOPS*(dfn1VectFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1VectFusNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1VectFusNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1VectFusNonProb_32mpi_4omp.GFLOPS*(dfn1VectFusNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1VectFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1VectFusProb_32mpi_4omp.GFLOPS*(dfn1VectFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4

fig.add_trace(go.Scatter(x=(dfn1VectFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1VectFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1VectFusNonProb_16mpi_8omp.GFLOPS*(dfn1VectFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1VectFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1VectFusProb_16mpi_8omp.GFLOPS*(dfn1VectFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))


# Case 5

fig.add_trace(go.Scatter(x=(dfn1VectFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1VectFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1VectFusNonProb_8mpi_16omp.GFLOPS*(dfn1VectFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1VectFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1VectFusProb_8mpi_16omp.GFLOPS*(dfn1VectFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Fused-VECT: Probmpi vs Non-Probempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 3) NonFused_VECT_Probmpi 

### Adding Columns

In [58]:
dfn1_vect_nonfused_probempi["N_Nodes"] = 1
dfn1_vect_nonfused_probempi['NPX'] = (dfn1_vect_nonfused_probempi.GLB_X//dfn1_vect_nonfused_probempi.LOC_X)
dfn1_vect_nonfused_probempi['NPY'] = (dfn1_vect_nonfused_probempi.GLB_Y//dfn1_vect_nonfused_probempi.LOC_Y)
dfn1_vect_nonfused_probempi['NPZ'] = (dfn1_vect_nonfused_probempi.GLB_Z//dfn1_vect_nonfused_probempi.LOC_Z)
dfn1_vect_nonfused_probempi['NPT'] = (dfn1_vect_nonfused_probempi.GLB_T//dfn1_vect_nonfused_probempi.LOC_T)
dfn1_vect_nonfused_probempi["Total_proc"] = dfn1_vect_nonfused_probempi["omp_thrd"]* dfn1_vect_nonfused_probempi.MPI_size
# dfn1_vect_nonfused_probempi.columns

### Data Summary

In [59]:
dfn1_vect_nonfused_probempi_summary = dfn1_vect_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_vect_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,35
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [60]:
dfn1VectNonFusProb_8mpi_16omp = dfn1_vect_nonfused_probempi[(dfn1_vect_nonfused_probempi.omp_thrd == 16) & (dfn1_vect_nonfused_probempi.GLB_T/dfn1_vect_nonfused_probempi.LOC_T == 1) & (dfn1_vect_nonfused_probempi.GLB_X/dfn1_vect_nonfused_probempi.LOC_X == 2) & (dfn1_vect_nonfused_probempi.GLB_Y/dfn1_vect_nonfused_probempi.LOC_Y == 2)& (dfn1_vect_nonfused_probempi.GLB_Z/dfn1_vect_nonfused_probempi.LOC_Z == 2)]
dfn1VectNonFusProb_16mpi_8omp = dfn1_vect_nonfused_probempi[(dfn1_vect_nonfused_probempi.omp_thrd == 8) & (dfn1_vect_nonfused_probempi.GLB_T/dfn1_vect_nonfused_probempi.LOC_T == 2) & (dfn1_vect_nonfused_probempi.GLB_X/dfn1_vect_nonfused_probempi.LOC_X == 2) & (dfn1_vect_nonfused_probempi.GLB_Y/dfn1_vect_nonfused_probempi.LOC_Y == 2)& (dfn1_vect_nonfused_probempi.GLB_Z/dfn1_vect_nonfused_probempi.LOC_Z == 2)]
dfn1VectNonFusProb_32mpi_4omp = dfn1_vect_nonfused_probempi[(dfn1_vect_nonfused_probempi.omp_thrd == 4) & (dfn1_vect_nonfused_probempi.GLB_T/dfn1_vect_nonfused_probempi.LOC_T == 4) & (dfn1_vect_nonfused_probempi.GLB_X/dfn1_vect_nonfused_probempi.LOC_X == 2) & (dfn1_vect_nonfused_probempi.GLB_Y/dfn1_vect_nonfused_probempi.LOC_Y == 2)& (dfn1_vect_nonfused_probempi.GLB_Z/dfn1_vect_nonfused_probempi.LOC_Z == 2)]
dfn1VectNonFusProb_64mpi_2omp = dfn1_vect_nonfused_probempi[(dfn1_vect_nonfused_probempi.omp_thrd == 2) & (dfn1_vect_nonfused_probempi.GLB_T/dfn1_vect_nonfused_probempi.LOC_T == 8) & (dfn1_vect_nonfused_probempi.GLB_X/dfn1_vect_nonfused_probempi.LOC_X == 2) & (dfn1_vect_nonfused_probempi.GLB_Y/dfn1_vect_nonfused_probempi.LOC_Y == 2)& (dfn1_vect_nonfused_probempi.GLB_Z/dfn1_vect_nonfused_probempi.LOC_Z == 2)]
dfn1VectNonFusProb_128mpi_1omp = dfn1_vect_nonfused_probempi[(dfn1_vect_nonfused_probempi.omp_thrd == 1) & (dfn1_vect_nonfused_probempi.GLB_T/dfn1_vect_nonfused_probempi.LOC_T == 16) & (dfn1_vect_nonfused_probempi.GLB_X/dfn1_vect_nonfused_probempi.LOC_X == 2) & (dfn1_vect_nonfused_probempi.GLB_Y/dfn1_vect_nonfused_probempi.LOC_Y == 2)& (dfn1_vect_nonfused_probempi.GLB_Z/dfn1_vect_nonfused_probempi.LOC_Z == 2)]


## 4) NonFused_VECT_NonProbmpi  

### Adding Columns

In [61]:
dfn1_vect_nonfused_nonprobempi["N_Nodes"] = 1
dfn1_vect_nonfused_nonprobempi['NPX'] = (dfn1_vect_nonfused_nonprobempi.GLB_X//dfn1_vect_nonfused_nonprobempi.LOC_X)
dfn1_vect_nonfused_nonprobempi['NPY'] = (dfn1_vect_nonfused_nonprobempi.GLB_Y//dfn1_vect_nonfused_nonprobempi.LOC_Y)
dfn1_vect_nonfused_nonprobempi['NPZ'] = (dfn1_vect_nonfused_nonprobempi.GLB_Z//dfn1_vect_nonfused_nonprobempi.LOC_Z)
dfn1_vect_nonfused_nonprobempi['NPT'] = (dfn1_vect_nonfused_nonprobempi.GLB_T//dfn1_vect_nonfused_nonprobempi.LOC_T)
dfn1_vect_nonfused_nonprobempi["Total_proc"] = dfn1_vect_nonfused_nonprobempi["omp_thrd"]* dfn1_vect_nonfused_nonprobempi.MPI_size
# dfn1_vect_nonfused_nonprobempi.columns

### Data Summary

In [62]:
dfn1_vect_nonfused_nonprobempi_summary = dfn1_vect_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_vect_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,6


### Cases Setup

In [63]:
dfn1VectNonFusNonProb_8mpi_16omp = dfn1_vect_nonfused_nonprobempi[(dfn1_vect_nonfused_nonprobempi.omp_thrd == 16) & (dfn1_vect_nonfused_nonprobempi.GLB_T/dfn1_vect_nonfused_nonprobempi.LOC_T == 1) & (dfn1_vect_nonfused_nonprobempi.GLB_X/dfn1_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn1_vect_nonfused_nonprobempi.GLB_Y/dfn1_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_vect_nonfused_nonprobempi.GLB_Z/dfn1_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn1VectNonFusNonProb_16mpi_8omp = dfn1_vect_nonfused_nonprobempi[(dfn1_vect_nonfused_nonprobempi.omp_thrd == 8) & (dfn1_vect_nonfused_nonprobempi.GLB_T/dfn1_vect_nonfused_nonprobempi.LOC_T == 2) & (dfn1_vect_nonfused_nonprobempi.GLB_X/dfn1_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn1_vect_nonfused_nonprobempi.GLB_Y/dfn1_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_vect_nonfused_nonprobempi.GLB_Z/dfn1_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn1VectNonFusNonProb_32mpi_4omp = dfn1_vect_nonfused_nonprobempi[(dfn1_vect_nonfused_nonprobempi.omp_thrd == 4) & (dfn1_vect_nonfused_nonprobempi.GLB_T/dfn1_vect_nonfused_nonprobempi.LOC_T == 4) & (dfn1_vect_nonfused_nonprobempi.GLB_X/dfn1_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn1_vect_nonfused_nonprobempi.GLB_Y/dfn1_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_vect_nonfused_nonprobempi.GLB_Z/dfn1_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn1VectNonFusNonProb_64mpi_2omp = dfn1_vect_nonfused_nonprobempi[(dfn1_vect_nonfused_nonprobempi.omp_thrd == 2) & (dfn1_vect_nonfused_nonprobempi.GLB_T/dfn1_vect_nonfused_nonprobempi.LOC_T == 8) & (dfn1_vect_nonfused_nonprobempi.GLB_X/dfn1_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn1_vect_nonfused_nonprobempi.GLB_Y/dfn1_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_vect_nonfused_nonprobempi.GLB_Z/dfn1_vect_nonfused_nonprobempi.LOC_Z == 2)]
dfn1VectNonFusNonProb_128mpi_1omp = dfn1_vect_nonfused_nonprobempi[(dfn1_vect_nonfused_nonprobempi.omp_thrd == 1) & (dfn1_vect_nonfused_nonprobempi.GLB_T/dfn1_vect_nonfused_nonprobempi.LOC_T == 16) & (dfn1_vect_nonfused_nonprobempi.GLB_X/dfn1_vect_nonfused_nonprobempi.LOC_X == 2) & (dfn1_vect_nonfused_nonprobempi.GLB_Y/dfn1_vect_nonfused_nonprobempi.LOC_Y == 2)& (dfn1_vect_nonfused_nonprobempi.GLB_Z/dfn1_vect_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - NonFused-VECT: Probempi vs NonProbempi

In [64]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1VectNonFusNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1VectNonFusNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1VectNonFusNonProb_128mpi_1omp.GFLOPS*(dfn1VectNonFusNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1VectNonFusProb_128mpi_1omp.Dirc_Data_Mov*(dfn1VectNonFusProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1VectNonFusProb_128mpi_1omp.GFLOPS*(dfn1VectNonFusProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1VectNonFusNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1VectNonFusNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1VectNonFusNonProb_64mpi_2omp.GFLOPS*(dfn1VectNonFusNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1VectNonFusProb_64mpi_2omp.Dirc_Data_Mov*(dfn1VectNonFusProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1VectNonFusProb_64mpi_2omp.GFLOPS*(dfn1VectNonFusProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1VectNonFusNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1VectNonFusNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1VectNonFusNonProb_32mpi_4omp.GFLOPS*(dfn1VectNonFusNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectNonFusProb_32mpi_4omp.Dirc_Data_Mov*(dfn1VectNonFusProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1VectNonFusProb_32mpi_4omp.GFLOPS*(dfn1VectNonFusProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4

fig.add_trace(go.Scatter(x=(dfn1VectNonFusNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1VectNonFusNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1VectNonFusNonProb_16mpi_8omp.GFLOPS*(dfn1VectNonFusNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectNonFusProb_16mpi_8omp.Dirc_Data_Mov*(dfn1VectNonFusProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1VectNonFusProb_16mpi_8omp.GFLOPS*(dfn1VectNonFusProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))


# Case 5

fig.add_trace(go.Scatter(x=(dfn1VectNonFusNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1VectNonFusNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1VectNonFusNonProb_8mpi_16omp.GFLOPS*(dfn1VectNonFusNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectNonFusProb_8mpi_16omp.Dirc_Data_Mov*(dfn1VectNonFusProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1VectNonFusProb_8mpi_16omp.GFLOPS*(dfn1VectNonFusProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNonFusProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-VECT: Probmpi vs Non-Probempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))

## 5) Newgeo_VECT_Probmpi 

### Adding Columns

In [65]:
dfn1_vect_newgeo_probempi["N_Nodes"] = 1
dfn1_vect_newgeo_probempi['NPX'] = (dfn1_vect_newgeo_probempi.GLB_X//dfn1_vect_newgeo_probempi.LOC_X)
dfn1_vect_newgeo_probempi['NPY'] = (dfn1_vect_newgeo_probempi.GLB_Y//dfn1_vect_newgeo_probempi.LOC_Y)
dfn1_vect_newgeo_probempi['NPZ'] = (dfn1_vect_newgeo_probempi.GLB_Z//dfn1_vect_newgeo_probempi.LOC_Z)
dfn1_vect_newgeo_probempi['NPT'] = (dfn1_vect_newgeo_probempi.GLB_T//dfn1_vect_newgeo_probempi.LOC_T)
dfn1_vect_newgeo_probempi["Total_proc"] = dfn1_vect_newgeo_probempi["omp_thrd"]* dfn1_vect_newgeo_probempi.MPI_size
# dfn1_vect_newgeo_probempi.columns

### Data Summary

In [66]:
dfn1_vect_newgeo_probempi_summary = dfn1_vect_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_vect_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,70
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,6


### Cases Setup

In [67]:
dfn1VectNewgeoProb_8mpi_16omp = dfn1_vect_newgeo_probempi[(dfn1_vect_newgeo_probempi.omp_thrd == 16) & (dfn1_vect_newgeo_probempi.GLB_T/dfn1_vect_newgeo_probempi.LOC_T == 1) & (dfn1_vect_newgeo_probempi.GLB_X/dfn1_vect_newgeo_probempi.LOC_X == 2) & (dfn1_vect_newgeo_probempi.GLB_Y/dfn1_vect_newgeo_probempi.LOC_Y == 2)& (dfn1_vect_newgeo_probempi.GLB_Z/dfn1_vect_newgeo_probempi.LOC_Z == 2)]
dfn1VectNewgeoProb_16mpi_8omp = dfn1_vect_newgeo_probempi[(dfn1_vect_newgeo_probempi.omp_thrd == 8) & (dfn1_vect_newgeo_probempi.GLB_T/dfn1_vect_newgeo_probempi.LOC_T == 2) & (dfn1_vect_newgeo_probempi.GLB_X/dfn1_vect_newgeo_probempi.LOC_X == 2) & (dfn1_vect_newgeo_probempi.GLB_Y/dfn1_vect_newgeo_probempi.LOC_Y == 2)& (dfn1_vect_newgeo_probempi.GLB_Z/dfn1_vect_newgeo_probempi.LOC_Z == 2)]
dfn1VectNewgeoProb_32mpi_4omp = dfn1_vect_newgeo_probempi[(dfn1_vect_newgeo_probempi.omp_thrd == 4) & (dfn1_vect_newgeo_probempi.GLB_T/dfn1_vect_newgeo_probempi.LOC_T == 4) & (dfn1_vect_newgeo_probempi.GLB_X/dfn1_vect_newgeo_probempi.LOC_X == 2) & (dfn1_vect_newgeo_probempi.GLB_Y/dfn1_vect_newgeo_probempi.LOC_Y == 2)& (dfn1_vect_newgeo_probempi.GLB_Z/dfn1_vect_newgeo_probempi.LOC_Z == 2)]
dfn1VectNewgeoProb_64mpi_2omp = dfn1_vect_newgeo_probempi[(dfn1_vect_newgeo_probempi.omp_thrd == 2) & (dfn1_vect_newgeo_probempi.GLB_T/dfn1_vect_newgeo_probempi.LOC_T == 8) & (dfn1_vect_newgeo_probempi.GLB_X/dfn1_vect_newgeo_probempi.LOC_X == 2) & (dfn1_vect_newgeo_probempi.GLB_Y/dfn1_vect_newgeo_probempi.LOC_Y == 2)& (dfn1_vect_newgeo_probempi.GLB_Z/dfn1_vect_newgeo_probempi.LOC_Z == 2)]
dfn1VectNewgeoProb_128mpi_1omp = dfn1_vect_newgeo_probempi[(dfn1_vect_newgeo_probempi.omp_thrd == 1) & (dfn1_vect_newgeo_probempi.GLB_T/dfn1_vect_newgeo_probempi.LOC_T == 16) & (dfn1_vect_newgeo_probempi.GLB_X/dfn1_vect_newgeo_probempi.LOC_X == 2) & (dfn1_vect_newgeo_probempi.GLB_Y/dfn1_vect_newgeo_probempi.LOC_Y == 2)& (dfn1_vect_newgeo_probempi.GLB_Z/dfn1_vect_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_VECT_NonProbmpi  

### Adding Columns

In [68]:
dfn1_vect_newgeo_nonprobempi["N_Nodes"] = 1
dfn1_vect_newgeo_nonprobempi['NPX'] = (dfn1_vect_newgeo_nonprobempi.GLB_X//dfn1_vect_newgeo_nonprobempi.LOC_X)
dfn1_vect_newgeo_nonprobempi['NPY'] = (dfn1_vect_newgeo_nonprobempi.GLB_Y//dfn1_vect_newgeo_nonprobempi.LOC_Y)
dfn1_vect_newgeo_nonprobempi['NPZ'] = (dfn1_vect_newgeo_nonprobempi.GLB_Z//dfn1_vect_newgeo_nonprobempi.LOC_Z)
dfn1_vect_newgeo_nonprobempi['NPT'] = (dfn1_vect_newgeo_nonprobempi.GLB_T//dfn1_vect_newgeo_nonprobempi.LOC_T)
dfn1_vect_newgeo_nonprobempi["Total_proc"] = dfn1_vect_newgeo_nonprobempi["omp_thrd"]* dfn1_vect_newgeo_nonprobempi.MPI_size
# dfn1_vect_newgeo_nonprobempi.columns

### Data Summary

In [69]:
dfn1_vect_newgeo_nonprobempi_summary = dfn1_vect_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn1_vect_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,8,16,1,2,2,2,128,71
1,16,8,2,2,2,2,128,71
2,32,4,4,2,2,2,128,36
3,64,2,8,2,2,2,128,17
4,128,1,16,2,2,2,128,7


### Cases Setup

In [70]:
dfn1VectNewgeoNonProb_8mpi_16omp = dfn1_vect_newgeo_nonprobempi[(dfn1_vect_newgeo_nonprobempi.omp_thrd == 16) & (dfn1_vect_newgeo_nonprobempi.GLB_T/dfn1_vect_newgeo_nonprobempi.LOC_T == 1) & (dfn1_vect_newgeo_nonprobempi.GLB_X/dfn1_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn1_vect_newgeo_nonprobempi.GLB_Y/dfn1_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_vect_newgeo_nonprobempi.GLB_Z/dfn1_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn1VectNewgeoNonProb_16mpi_8omp = dfn1_vect_newgeo_nonprobempi[(dfn1_vect_newgeo_nonprobempi.omp_thrd == 8) & (dfn1_vect_newgeo_nonprobempi.GLB_T/dfn1_vect_newgeo_nonprobempi.LOC_T == 2) & (dfn1_vect_newgeo_nonprobempi.GLB_X/dfn1_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn1_vect_newgeo_nonprobempi.GLB_Y/dfn1_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_vect_newgeo_nonprobempi.GLB_Z/dfn1_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn1VectNewgeoNonProb_32mpi_4omp = dfn1_vect_newgeo_nonprobempi[(dfn1_vect_newgeo_nonprobempi.omp_thrd == 4) & (dfn1_vect_newgeo_nonprobempi.GLB_T/dfn1_vect_newgeo_nonprobempi.LOC_T == 4) & (dfn1_vect_newgeo_nonprobempi.GLB_X/dfn1_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn1_vect_newgeo_nonprobempi.GLB_Y/dfn1_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_vect_newgeo_nonprobempi.GLB_Z/dfn1_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn1VectNewgeoNonProb_64mpi_2omp = dfn1_vect_newgeo_nonprobempi[(dfn1_vect_newgeo_nonprobempi.omp_thrd == 2) & (dfn1_vect_newgeo_nonprobempi.GLB_T/dfn1_vect_newgeo_nonprobempi.LOC_T == 8) & (dfn1_vect_newgeo_nonprobempi.GLB_X/dfn1_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn1_vect_newgeo_nonprobempi.GLB_Y/dfn1_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_vect_newgeo_nonprobempi.GLB_Z/dfn1_vect_newgeo_nonprobempi.LOC_Z == 2)]
dfn1VectNewgeoNonProb_128mpi_1omp = dfn1_vect_newgeo_nonprobempi[(dfn1_vect_newgeo_nonprobempi.omp_thrd == 1) & (dfn1_vect_newgeo_nonprobempi.GLB_T/dfn1_vect_newgeo_nonprobempi.LOC_T == 16) & (dfn1_vect_newgeo_nonprobempi.GLB_X/dfn1_vect_newgeo_nonprobempi.LOC_X == 2) & (dfn1_vect_newgeo_nonprobempi.GLB_Y/dfn1_vect_newgeo_nonprobempi.LOC_Y == 2)& (dfn1_vect_newgeo_nonprobempi.GLB_Z/dfn1_vect_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo VECT: Probempi vs NonProbempi

In [71]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn1VectNewgeoNonProb_128mpi_1omp.Dirc_Data_Mov*(dfn1VectNewgeoNonProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1VectNewgeoNonProb_128mpi_1omp.GFLOPS*(dfn1VectNewgeoNonProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_128mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn1VectNewgeoProb_128mpi_1omp.Dirc_Data_Mov*(dfn1VectNewgeoProb_128mpi_1omp.MPI_size)), 
                         y=(dfn1VectNewgeoProb_128mpi_1omp.GFLOPS*(dfn1VectNewgeoProb_128mpi_1omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_128mpi_1omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn1VectNewgeoNonProb_64mpi_2omp.Dirc_Data_Mov*(dfn1VectNewgeoNonProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1VectNewgeoNonProb_64mpi_2omp.GFLOPS*(dfn1VectNewgeoNonProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_64mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn1VectNewgeoProb_64mpi_2omp.Dirc_Data_Mov*(dfn1VectNewgeoProb_64mpi_2omp.MPI_size)), 
                         y=(dfn1VectNewgeoProb_64mpi_2omp.GFLOPS*(dfn1VectNewgeoProb_64mpi_2omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_64mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn1VectNewgeoNonProb_32mpi_4omp.Dirc_Data_Mov*(dfn1VectNewgeoNonProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1VectNewgeoNonProb_32mpi_4omp.GFLOPS*(dfn1VectNewgeoNonProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_32mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectNewgeoProb_32mpi_4omp.Dirc_Data_Mov*(dfn1VectNewgeoProb_32mpi_4omp.MPI_size)), 
                         y=(dfn1VectNewgeoProb_32mpi_4omp.GFLOPS*(dfn1VectNewgeoProb_32mpi_4omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_32mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))



# Case 4

fig.add_trace(go.Scatter(x=(dfn1VectNewgeoNonProb_16mpi_8omp.Dirc_Data_Mov*(dfn1VectNewgeoNonProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1VectNewgeoNonProb_16mpi_8omp.GFLOPS*(dfn1VectNewgeoNonProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_16mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='red',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectNewgeoProb_16mpi_8omp.Dirc_Data_Mov*(dfn1VectNewgeoProb_16mpi_8omp.MPI_size)), 
                         y=(dfn1VectNewgeoProb_16mpi_8omp.GFLOPS*(dfn1VectNewgeoProb_16mpi_8omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_16mpi_8omp',
                         line=dict(color='green',width=3),
                         marker=dict(symbol='circle',size=5,line=dict(color='green',width=2))))


# Case 5

fig.add_trace(go.Scatter(x=(dfn1VectNewgeoNonProb_8mpi_16omp.Dirc_Data_Mov*(dfn1VectNewgeoNonProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1VectNewgeoNonProb_8mpi_16omp.GFLOPS*(dfn1VectNewgeoNonProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoNonProb_8mpi_16omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='chocolate',width=2))))


fig.add_trace(go.Scatter(x=(dfn1VectNewgeoProb_8mpi_16omp.Dirc_Data_Mov*(dfn1VectNewgeoProb_8mpi_16omp.MPI_size)), 
                         y=(dfn1VectNewgeoProb_8mpi_16omp.GFLOPS*(dfn1VectNewgeoProb_8mpi_16omp.MPI_size)), 
                         mode='lines+markers', 
                         name='VectNewgeoProb_8mpi_16omp',
                         line=dict(color='darkviolet',width=3),
                         marker=dict(symbol='triangle-up',size=4,line=dict(color='crimson',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo-VECT: Probmpi vs Non-Probempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1))